  COMPLEXES DATA

In [1]:
import requests

url = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key=Wsd3stMv53a8qB99rszTUTOScJTU2EhP5V93nlfy"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{"generated_at":"2025-02-01T14:36:10+00:00","complexes":[{"id":"sr:complex:705","name":"Nacional","venues":[{"id":"sr:venue:70045","name":"Cancha Central","city_name":"Santiago","country_name":"Chile","country_code":"CHL","timezone":"America\/Santiago"}]},{"id":"sr:complex:1078","name":"Estadio de la Cartuja","venues":[{"id":"sr:venue:74856","name":"Centre Court","city_name":"Seville","country_name":"Spain","country_code":"ESP","timezone":"Europe\/Madrid"},{"id":"sr:venue:74858","name":"Court One","city_name":"Seville","country_name":"Spain","country_code":"ESP","timezone":"Europe\/Madrid"}]},{"id":"sr:complex:1495","name":"Sibur Arena","venues":[{"id":"sr:venue:1496","name":"COURT 1","city_name":"Saint Petersburg","country_name":"Russia","country_code":"RUS","timezone":"Europe\/Moscow"},{"id":"sr:venue:1500","name":"CENTER COURT","city_name":"Saint Petersburg","country_name":"Russia","country_code":"RUS","timezone":"Europe\/Moscow"},{"id":"sr:venue:62149","name":"Sibur Arena","city_na

In [2]:
json_complxdata=response.json()

In [3]:
complx_data=[]

In [4]:
for complexes in json_complxdata["complexes"]:
     complexes_info = {
          "complex_id": complexes.get("id"),
          "complex_name": complexes.get("name")
        
     }
     complx_data.append(complexes_info)

In [5]:
import pandas as pd
df3=pd.DataFrame(complx_data)

In [6]:
df3.to_csv("Complexes.csv",index=False)

In [7]:
venue_data=[]

In [8]:
complexes = json_complxdata['complexes']

In [9]:
for complex_info in json_complxdata["complexes"]:
    for venue in complex_info.get('venues', []):
        venue_info = {
            "complex_id": complex_info['id'],
            
            "venue_id": venue['id'],
            "venue_name": venue['name'],
            "city_name": venue['city_name'],
            "country_name": venue['country_name'],
            "country_code": venue['country_code'],
            "timezone": venue['timezone']
        }
        venue_data.append(venue_info)

In [10]:
df4=pd.DataFrame(venue_data)

In [11]:
df4.to_csv("Venues.csv",index=False)

In [ ]:
pip install mysql-connector-python

In [2]:
import mysql.connector

connection= mysql.connector.connect(
 host= "localhost",
 user= "root",
 password="5666",
 database="tennis_new"
)

print(connection)
mycursor = connection.cursor()

In [4]:
!python.exe -m pip install --upgrade pip

In [5]:
mycursor.execute("""
CREATE TABLE complexes (
    complex_id VARCHAR(50)  PRIMARY KEY,
    complex_name VARCHAR(100));""")
    
connection.commit()



In [6]:
mycursor.execute("""
CREATE TABLE venue (
    venue_id varchar(50) PRIMARY KEY, 
    venue_name varchar(100),
    city_name varchar(100) ,
    country_name varchar(100) ,
    country_code varchar(100),
    timezone varchar(100),
    complex_id varchar(50),
    FOREIGN KEY (complex_id) REFERENCES complexes(complex_id));""")
connection.commit()

In [7]:
import pandas as pd


In [8]:
df3=pd.read_csv("Complexes.csv")
df4=pd.read_csv("Venues.csv")

In [9]:
insert = "INSERT INTO complexes (complex_id, complex_name) VALUES (%s, %s)"

for index, row in df3.iterrows():
    mycursor.execute(insert, (row['complex_id'], row['complex_name']))
connection.commit()

insert_venue = """
INSERT INTO venue (venue_id, venue_name, city_name, country_name, country_code, timezone, complex_id) 
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

for index, row in df4.iterrows():
    mycursor.execute(insert_venue, (
        row['venue_id'], 
        row['venue_name'], 
        row['city_name'], 
        row['country_name'], 
        row['country_code'], 
        row['timezone'], 
        row['complex_id']  
    ))
connection.commit()
print("success data")

success data


In [10]:
#1)List all venues along with their associated complex name
mycursor.execute("""SELECT venue.venue_name,complexes.complex_name FROM venue 
                 inner join complexes ON venue.complex_id=complexes.complex_id""")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))


+------------------------------------------------------------+------------------------------------------------------------+
| venue_name                                                 | complex_name                                               |
|------------------------------------------------------------+------------------------------------------------------------|
| Center Court                                               | Kindarena                                                  |
| Amelie Mauresmo                                            | Kindarena                                                  |
| Celine Dumerc                                              | Kindarena                                                  |
| Court 1                                                    | Kindarena                                                  |
| Centre Court                                               | Estadio de la Cartuja                                      |
| Court 

In [11]:
#2Count the number of venues in each complex
mycursor.execute(""" SELECT 
    Complexes.complex_name, 
    COUNT(Venue.venue_name) AS venue_count
FROM 
    Venue
JOIN 
    Complexes
ON 
    Venue.complex_id = Complexes.complex_id
GROUP BY 
    Complexes.complex_name order by venue_count ;
 """)
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+------------------------------------------------------------+---------------+
| complex_name                                               |   venue_count |
|------------------------------------------------------------+---------------|
| Garanti Koza Arena                                         |             1 |
| The O2                                                     |             1 |
| Stadium Goran Ivanisevic in ITC Stella Maris               |             1 |
| China National Tennis Center                               |             1 |
| Gerry Weber Stadium                                        |             1 |
| Swiss Tennis Arena                                         |             1 |
| Tokyo Metropolitan Gymnasium                               |             1 |
| Sanchez-Casal Tennis Academy                               |             1 |
| Palazzetto dello Sport                                     |             1 |
| Club Los Lagartos                                 

In [12]:
#3)Get details of venues in a specific country (e.g., Chile)
mycursor.execute("SELECT country_name,venue_id,venue_name,city_name,country_code,timezone from Venue WHERE country_name='Chile' ")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+----------------+----------------+-------------------------+--------------+----------------+------------------+
| country_name   | venue_id       | venue_name              | city_name    | country_code   | timezone         |
|----------------+----------------+-------------------------+--------------+----------------+------------------|
| Chile          | sr:venue:13830 | Cancha 1                | Santiago     | CHL            | America/Santiago |
| Chile          | sr:venue:13832 | Cancha 3                | Santiago     | CHL            | America/Santiago |
| Chile          | sr:venue:15858 | Centre Court            | Santiago     | CHL            | America/Santiago |
| Chile          | sr:venue:15860 | Court 16                | Santiago     | CHL            | America/Santiago |
| Chile          | sr:venue:15862 | Court 15                | Santiago     | CHL            | America/Santiago |
| Chile          | sr:venue:15864 | Court 11                | Santiago     | CHL            | Am

In [13]:
#4)Identify all venues and their timezones
mycursor.execute("SELECT venue_name, timezone FROM Venue ")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+------------------------------------------------------------+--------------------------------+
| venue_name                                                 | timezone                       |
|------------------------------------------------------------+--------------------------------|
| Estadio Monumental                                         | America/Lima                   |
| Stadium                                                    | Asia/Shanghai                  |
| Court 2                                                    | Asia/Shanghai                  |
| Court 7                                                    | Australia/Sydney               |
| Court 20                                                   | Australia/Melbourne            |
| Court 22                                                   | Australia/Melbourne            |
| Estadio Ivan Elias Moreno                                  | America/Lima                   |
| Stade Numa-Daly Magenta               

In [14]:
#5)Find complexes that have more than one venue
mycursor.execute("""SELECT complex_id FROM (SELECT COUNT(venue_id) as venue_id_count, complex_id FROM Venue GROUP BY complex_id) 
                 AS childquery
                  WHERE venue_id_count>1 """)
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+------------------+
| complex_id       |
|------------------|
| sr:complex:10063 |
| sr:complex:1078  |
| sr:complex:11769 |
| sr:complex:12241 |
| sr:complex:1495  |
| sr:complex:15014 |
| sr:complex:15371 |
| sr:complex:16020 |
| sr:complex:16612 |
| sr:complex:17945 |
| sr:complex:18221 |
| sr:complex:19737 |
| sr:complex:19951 |
| sr:complex:21910 |
| sr:complex:2375  |
| sr:complex:26983 |
| sr:complex:26985 |
| sr:complex:27093 |
| sr:complex:27095 |
| sr:complex:28457 |
| sr:complex:28617 |
| sr:complex:28926 |
| sr:complex:32327 |
| sr:complex:32812 |
| sr:complex:33186 |
| sr:complex:33188 |
| sr:complex:33190 |
| sr:complex:33192 |
| sr:complex:33196 |
| sr:complex:33198 |
| sr:complex:33200 |
| sr:complex:33206 |
| sr:complex:33208 |
| sr:complex:33210 |
| sr:complex:33214 |
| sr:complex:33218 |
| sr:complex:33222 |
| sr:complex:33228 |
| sr:complex:33230 |
| sr:complex:33236 |
| sr:complex:33238 |
| sr:complex:33240 |
| sr:complex:33242 |
| sr:complex:33244 |
| sr:complex:

In [15]:
#6)List venues grouped by country
mycursor.execute("SELECT country_name, GROUP_CONCAT(venue_name) AS venue FROM Venue GROUP BY country_name ;")
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))

+----------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------